## 准备数据

In [70]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [71]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [72]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        super(myModel, self).__init__()
        self.W1 = tf.Variable(tf.random.normal([28*28, 14*14]))
        self.b1 = tf.Variable(tf.zeros([14*14]))
        self.W2 = tf.Variable(tf.random.normal([14*14, 49]))
        self.b2 = tf.Variable(tf.zeros([49]))

        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        tmp_x = tf.matmul(flat_x, self.W1) + self.b1
        logits = tf.matmul(tmp_x, self.W2) + self.b2
        # logits = x
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [73]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [74]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 268.2334 ; accuracy 0.030266667
epoch 1 : loss 230.42125 ; accuracy 0.045333333
epoch 2 : loss 206.405 ; accuracy 0.056766666
epoch 3 : loss 187.878 ; accuracy 0.06646667
epoch 4 : loss 172.3404 ; accuracy 0.07751667
epoch 5 : loss 158.74698 ; accuracy 0.08895
epoch 6 : loss 146.69286 ; accuracy 0.102183335
epoch 7 : loss 136.0418 ; accuracy 0.11776666
epoch 8 : loss 126.7086 ; accuracy 0.13518333
epoch 9 : loss 118.629135 ; accuracy 0.15243334
epoch 10 : loss 111.6562 ; accuracy 0.16976666
epoch 11 : loss 105.60368 ; accuracy 0.18608333
epoch 12 : loss 100.29682 ; accuracy 0.20121667
epoch 13 : loss 95.598915 ; accuracy 0.21551667
epoch 14 : loss 91.394035 ; accuracy 0.22955
epoch 15 : loss 87.59444 ; accuracy 0.24241666
epoch 16 : loss 84.13686 ; accuracy 0.2543
epoch 17 : loss 80.97957 ; accuracy 0.26716667
epoch 18 : loss 78.08204 ; accuracy 0.27896667
epoch 19 : loss 75.412964 ; accuracy 0.28978333
epoch 20 : loss 72.94148 ; accuracy 0.30033332
epoch 21 : loss 70.64